# Let's Build GPT: from scratch
Following Andrej Karpathy's Tutorial:
https://www.youtube.com/watch?v=kCc8FmEb1nY

In [1]:
# We start by downloading the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# Read the file and print the length
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("Length of the dataset in characters: ", len(text))
print(text[:1000])

Length of the dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in 

In [3]:
# Finding all the unique characters in the original 'text' string
chars = sorted(list(set(text))) # set builds an unordered collection of unique elements
vocab_size = len(chars)
print(''.join(chars))
print("Vocabulary Size: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary Size:  65


### Using a Dictionary comprehension equivalent to:
```
stoi = {}
for i, ch in enumerate(chars):
    stoi[ch] = i
```

In [4]:
# Mapping from characters to integers and viceversa (encoding and decoding)
# TODO: Try different, more advanced ways of encoding (i.e: Google's "SentencePiece" or OpenAI's "tiktoken", etc)
stoi = { char:idx for idx,char in enumerate(chars) }
itos = { idx:char for idx,char in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]             # Encoder: Takes a string, outputs a list of integers
decode = lambda l: ''.join([itos[i] for i in l])    # Decoder: Takes a list of integers, outputs a string

print("Encoded:\n", encode("Hello there"))
print("Decoded:\n", decode(encode("Hello there")))

Encoded:
 [20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
Decoded:
 Hello there


In [5]:
# Encoding the entire 'text' dataset and storing it into a torch.Tensor
import torch # PyTorch: https://pytorch.org
import torch.nn as nn
from torch.nn import functional as F
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Data Shape: {data.shape}\tData Type: {data.dtype}")
print(data[:1000]) # The same 1000 characters we looked at earlier, but encoded

Data Shape: torch.Size([1115394])	Data Type: torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1,

In [6]:
# Splitting into training and validation sets (90/10)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8 # Sometimes also refered as "context length"
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]     # Inputs for the transformer
y = train_data[1:block_size+1]  # Targets for each position, offset by one

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When Input is {context}, Target is: {target}")

When Input is tensor([18]), Target is: 47
When Input is tensor([18, 47]), Target is: 56
When Input is tensor([18, 47, 56]), Target is: 57
When Input is tensor([18, 47, 56, 57]), Target is: 58
When Input is tensor([18, 47, 56, 57, 58]), Target is: 1
When Input is tensor([18, 47, 56, 57, 58,  1]), Target is: 15
When Input is tensor([18, 47, 56, 57, 58,  1, 15]), Target is: 47
When Input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), Target is: 58


In [9]:
torch.manual_seed(1337)
batch_size = 4 # How many independent equences will we process in parallel
block_size = 8 # The maximum context length for predictions

def get_batch(split):
    """ Generates a small batch of data of Inputs 'x' and Targets 'y'"""
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f"Inputs:\n{xb.shape}\n{xb}")
print(f"Targets:\n{yb.shape}\n{yb}")

print('--------')

for batch in range(batch_size):     # batch dimension
    for time in range(block_size):  # time dimension
        context = xb[batch, :time+1]
        target = yb[batch, time]
        print(f"When Input is {context.tolist()}, Target is: {target}")

Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------
When Input is [24], Target is: 43
When Input is [24, 43], Target is: 58
When Input is [24, 43, 58], Target is: 5
When Input is [24, 43, 58, 5], Target is: 57
When Input is [24, 43, 58, 5, 57], Target is: 1
When Input is [24, 43, 58, 5, 57, 1], Target is: 46
When Input is [24, 43, 58, 5, 57, 1, 46], Target is: 43
When Input is [24, 43, 58, 5, 57, 1, 46, 43], Target is: 39
When Input is [44], Target is: 53
When Input is [44, 53], Target is: 56
When Input is [44, 53, 56], Target is: 1
When Input is [44, 53, 56, 1], Target is: 58
When Input is [44, 53, 56, 1, 58], Target is: 46
When Input is [44

In [10]:
print(xb) # Our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookyp table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
       
    def forward(self, idx, targets=None):
        # idx and targets are both (B.T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) -> (Batch, Time, Channel) Channel is vocab_size = 65
        
        if targets is None:
            loss = None
        else:
            # Reformating the tensors so they are in the format that 'F.cross_entropy' expects
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, loss = self(idx)
            # Focus only in the last time step
            logits = logits[:, -1, :] # Becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            #Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [12]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
# Training the model
batch_size = 32
epochs = 10_000
for steps in range(epochs):
    
    # Sample a batch of data
    xb, yb = get_batch('train')
    
    # Evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.5727508068084717


In [14]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenou


```python
import torch
print(torch.__version__)
print(torch.cuda.get_device_name(0))
print(f"Cuda available: {torch.cuda.is_available()}")
```

In [15]:
# Consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # Batch, Time, Channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

### (1) Unefficient version because we're looping through the whole tensor with the for loops

In [19]:
# We want x[b,t] = mean:{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # Bag of Words initialized at 0
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # xprev.shape -> (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

### (2) Better version using Matrix multiplication

In [20]:
wei = torch.tril(torch.ones(T, T)) # wei -> Short for 'weights'
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2, atol=1e-6)

True

### (3) Softmax version

In [21]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # '-inf' -> Negative infinity
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3, atol=1e-6)

True

### (4) Self-Attention!
#### Notes:
- Attention is a **communication mechanism.** Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other.
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just meanns that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module).
- "Scaled" attention additionally divides wei by 1/sqrt(head_size). This makes it so when input Q, K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below.

In [25]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # Batch, Time, Channels
x = torch.randn(B,T,C)

# Implementing a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
# Compute attention scores ("affinities")
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
# Without this mask layer all nodes (tokens here) will be able to talk to each other like in an encoder block.
# Whereas in decoder block this will always be present.
wei = wei.masked_fill(tril == 0, float('-inf')) # '-inf' -> Negative infinity.
wei = F.softmax(wei, dim=1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

### Step by step toy version of 2nd version

In [9]:
torch.manual_seed(42)
a = torch.ones(3, 3)
a = torch.tril(a)
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b # '@' is for Matrix multiplication
print(f"a=\n{a}\n----------")
print(f"b=\n{b}\n----------")
print(f"c=\n{c}")

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
